In [5]:
#https://keras.io/examples/vision/3D_image_classification/#setup
#tried to use above tutorial

import os
import zipfile
import numpy as np
import tensorflow as tf
import pandas as pd
from scipy.io import loadmat
from scipy.io import savemat


from tensorflow import keras
from tensorflow.keras import layers

#Lucas directory
#C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Ian directory
#H:/Documents/Github repositories/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

dataFolder = 'H:/Documents/Github repositories/SE4450-project-code/Preprocessing/'


In [73]:
#Setting up the structure of the data to be fed into the neural network\
#Tried to convert lucas' code into functions and stuff, not totally sure of what the data structure is happening

#Getting data from .mat file
def readData(patientID):
    rawMatFile = loadmat(dataFolder+patientID+'.mat')
    return rawMatFile['rf1']

#Grabbing frame pair and label data from excel
def readExcel(filename, sheetname):
    #Read in label data
    dicLabeledData = pd.read_excel('Labels.xlsx', sheet_name=sheetname, engine='openpyxl')
    
    vecFirstFrames = dicLabeledData['Frame 1'].tolist()
    vecSecondFrames = dicLabeledData['Frame 2'].tolist()
    vecLabelValue = dicLabeledData['Label axial'].tolist()

    matFramePairLabels = [vecFirstFrames,vecSecondFrames,vecLabelValue]

    return matFramePairLabels

def loadPairsAndLabels(patientID):
    rawData = readData(patientID)
    matFramePairLabels = readExcel('Labels.xlsx', patientID)

    stackedLabeledPairs = []

    for pairNum in range(len(matFramePairLabels[0])):
        firstFrame = matFramePairLabels[0][pairNum]
        secondFrame = matFramePairLabels[1][pairNum]
        firstFrameData = rawData[:len(rawData)][:len(rawData[0])][firstFrame]
        secondFrameData = rawData[:len(rawData)][:len(rawData[0])][secondFrame]
        stackedFrame = np.stack((firstFrameData,secondFrameData), -1)

    return stackedFrame



In [74]:
#Read in label data
stackedLabeledPairs = loadPairsAndLabels('P90-W0-S4')
matFramePairLabels = readExcel('Labels.xlsx', 'P90-W0-S4')

print(stackedLabeledPairs)

[[[ -81   55]
  [  58  -15]
  [  45  -32]
  ...
  [ -37    3]
  [ -33   11]
  [ -29    6]]

 [[   2  -53]
  [ -23   35]
  [  14  -37]
  ...
  [  18  -12]
  [ -24    6]
  [   4  -21]]

 [[-127  175]
  [  -4    8]
  [  64  -99]
  ...
  [ -19   -2]
  [  15  -11]
  [ -17    0]]

 ...

 [[ -18   -8]
  [  46  -63]
  [  -9   32]
  ...
  [   5  -26]
  [  13  -18]
  [  -1    5]]

 [[  15   17]
  [ -17   16]
  [ -11    0]
  ...
  [ -32   12]
  [ -32    9]
  [ -17  -14]]

 [[  -2   -9]
  [ -19   -4]
  [   9   -2]
  ...
  [  -1   -6]
  [  -2    2]
  [  -1    7]]]


In [ ]:
def normalize(volume):
    #Adjust to normalize the scans to be between 0 and 1
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


In [ ]:
# Read and process the scans.
# Each scan is resized across height, width, and depth and rescaled.
bad_pairs = np.array([process_scan(path) for path in abnormal_scan_paths])
good_pairs = np.array([process_scan(path) for path in normal_scan_paths])

# For the good pairs
# assign 1, for the bad ones assign 0.
bad_labels = np.array([0 for _ in range(len(bad_pairs))])
good_labels = np.array([1 for _ in range(len(good_pairs))])

# Split data in the ratio 70-30 for training and validation.
x_train = np.concatenate((bad_pairs[:70], good_pairs[:70]), axis=0)
y_train = np.concatenate((bad_labels[:70], good_labels[:70]), axis=0)
x_val = np.concatenate((bad_pairs[70:], good_pairs[70:]), axis=0)
y_val = np.concatenate((bad_labels[70:], good_labels[70:]), axis=0)
print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)


In [ ]:
def train_preprocessing(volume, label):
    #"""Process training data by adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    #"""Process validation data by adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)


In [1]:
def get_model(width=128, height=128, depth=64):
    #"""Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()


NameError: name 'keras' is not defined

In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)